# Capacity and quantum geometry of parametrized quantum circuits


"Capacity and quantum geometry of parametrized quantum circuits" by T. Haug, K. Bharti, M.S. Kim

Example code to calculate quantum Fisher information matrix,effective quantum dimension and quantum natural gradient.

Calculates properties of a parametrized quantum circuit 
U(\theta)=W_L R_L(\theta) W_{L-1} R_{L-1}(\theta) ... W_1 R_1(\theta) \sqrt{H}\ket{0}

W_l is entangling layer of two-qubit entangling operations, R_l are single-qubit rotations

Based on qutip

@author: Tobias Haug, github txhaug
Imperial College London
Contact at tobiasxhaug@gmail.com


In [1]:
import qutip as qt

from functools import partial

import operator
from functools import reduce
import numpy as np

import scipy

In [2]:
def prod(factors):
    return reduce(operator.mul, factors, 1)


def flatten(l):
    return [item for sublist in l for item in sublist]

#tensors operators together 
def genFockOp(op,position,size,levels=2,opdim=0):
    opList=[qt.qeye(levels) for x in range(size-opdim)]
    opList[position]=op
    return qt.tensor(opList)

Set parameters here for circuit

In [3]:

n_qubits=4 #number qubits
depth=3 #number of layers

#type of entangling gate used
type_entanglers=0 #0: CNOT, 1:CPHASE, 2: \sqrt{iSWAP}

#how to arrange the entangling layer
entangling_arrangement=0 #0: one-dimensional nearest-neighbor CHAIN, 1: ALl-to-ALL connections


In [4]:

#random generator used
rng = np.random.default_rng(0)


#define angles for circuit
ini_angles=rng.random([depth,n_qubits])*2*np.pi

#define rotations for circuit in each layer, 1: X, 2:Y 3:Z
ini_pauli=rng.integers(1,4,[depth,n_qubits])

n_parameters=depth*n_qubits #number of parameters of circuit


cutoff_eigvals=10**-12 #define all eigenvalues of quantum fisher information metric as 0

#operators for circuit
levels=2#
opZ=[genFockOp(qt.sigmaz(),i,n_qubits,levels) for i in range(n_qubits)]
opX=[genFockOp(qt.sigmax(),i,n_qubits,levels) for i in range(n_qubits)]
opY=[genFockOp(qt.sigmay(),i,n_qubits,levels) for i in range(n_qubits)]
opId=genFockOp(qt.qeye(levels),0,n_qubits)
    

H=opZ[0]*opZ[1] #local Hamiltonian to calculate energy and gradient from
    

#define entangling gate arrangement, 
if(entangling_arrangement==0):    #here is for chain topology
    entangling_gate_index=[[2*j,2*j+1] for j in range(n_qubits//2)]+[[2*j+1,2*j+2] for j in range((n_qubits-1)//2)]
elif(entangling_arrangement==1):##all-to-all
    #randomize control and target for more entangling power for CNOT
    entangling_gate_index=flatten([[rng.permutation([i,j]) for j in range(i+1,n_qubits)] for i in range(n_qubits-1)])


#type of entangliers used
if(type_entanglers==0):#CNOT
    entangling_layer=prod([qt.qip.operations.cnot(n_qubits,j,k) for j,k in entangling_gate_index][::-1])#need [::-1] to invert order so that unitaries are multiplied in correct order
elif(type_entanglers==1):#CPHASE
    entangling_layer=prod([qt.qip.operations.csign(n_qubits,j,k) for j,k in entangling_gate_index][::-1])
elif(type_entanglers==2):#\sqrt{iSWAP}
    entangling_layer=prod([qt.qip.operations.sqrtiswap(n_qubits,[j,k]) for j,k in entangling_gate_index][::-1])



In [5]:
#calculate state and gradients

#list of values of gradient
gradient_list=np.zeros(n_parameters)

#save here quantum state of gradient
grad_state_list=[]


#p goes from -1 to n_parameters-1. -1 is to calculate quantum state, rest for gradient
for p in range(-1,n_parameters):
    counter=-1
    initial_state=qt.tensor([qt.basis(levels,0) for i in range(n_qubits)])
    #initial layer of fixed \sqrt{H} rotations
    initial_state=qt.tensor([qt.qip.operations.ry(np.pi/4) for i in range(n_qubits)])*initial_state
    
    #go through depth layers
    for j in range(depth):
        rot_op=[]
        #define parametrized single-qubit rotations at layer j
        for k in range(n_qubits):
            angle=ini_angles[j][k]
            if(ini_pauli[j][k]==1):
                rot_op.append(qt.qip.operations.rx(angle))
            elif(ini_pauli[j][k]==2):
                rot_op.append(qt.qip.operations.ry(angle))
            elif(ini_pauli[j][k]==3):
                rot_op.append(qt.qip.operations.rz(angle))
                
                
            #multiply in derivative of parametrized single-qubit rotation gate at layer j for parameter of circuit p
            #this is the exact derivative
            if(counter==p):
                if(ini_pauli[j][k]==1):
                    initial_state=(-1j*opX[k]/2)*initial_state
                elif(ini_pauli[j][k]==2):
                    initial_state=(-1j*opY[k]/2)*initial_state
                elif(ini_pauli[j][k]==3):
                    initial_state=(-1j*opZ[k]/2)*initial_state
                
            counter+=1
                
        #multiply in single-qbuit rotations 
        initial_state=qt.tensor(rot_op)*initial_state
    
        #add entangling layer
        initial_state=entangling_layer*initial_state
     
    if(p==-1):
        circuit_state=qt.Qobj(initial_state)#state generated by circuit
        energy=qt.expect(H,circuit_state)
        print("Energy of state",energy)

    else:
        grad_state_list.append(qt.Qobj(initial_state))#state with gradient applied for p-th parameter

        #gradient of circuit
        gradient_list[p]=2*np.real(circuit_state.overlap(H*initial_state))

Energy of state 0.1767766952966368


In [6]:
#quantum fisher information metric
#calculated as \text{Re}(\braket{\partial_i \psi}{\partial_j \psi}-\braket{\partial_i \psi}{\psi}\braket{\psi}{\partial_j \psi})

#first, calculate elements \braket{\psi}{\partial_j \psi})
single_qfi_elements=np.zeros(n_parameters,dtype=np.complex128)
for p in range(n_parameters):
    #print(circuit_state.overlap(grad_state_list[p]))
    single_qfi_elements[p]=circuit_state.overlap(grad_state_list[p])
            

#calculcate the qfi matrix
qfi_matrix=np.zeros([n_parameters,n_parameters])
for p in range(n_parameters):
    for q in range(p,n_parameters):
        qfi_matrix[p,q]=np.real(grad_state_list[p].overlap(grad_state_list[q])-np.conjugate(single_qfi_elements[p])*single_qfi_elements[q])
    
    
#use fact that qfi matrix is real and hermitian
for p in range(n_parameters):
    for q in range(p+1,n_parameters):  
        qfi_matrix[q,p]=qfi_matrix[p,q]

Get eigenvalues and eigenvectors of quantum fisher information metric. Gives information about the parameter space of the circuit. Zero eigenvalues indicate redundant parameters, i.e. which do not contribute The larger the eigenvalue, the more the quantum state changes when changing the parameters in the direction of the eigenvector-
For random initial parameters, the effective quantum dimension is equivalent to the parameter dimension, which is the total number of independent parameters that the quantum state generated by the quantum circuit can represent.

In [7]:
#get eigenvalues and eigenvectors of QFI
eigvals,eigvecs=scipy.linalg.eigh(qfi_matrix)

In [8]:
#non-zero eigenvalues of qfi
nonzero_eigvals=eigvals[eigvals>cutoff_eigvals]

#effective quantum dimension, i.e. number of independent directions when perturbing system
#is equivalent to parameter dimension (number of independent parameters of quantum state that can be represented by circuit) for random initial values of circuit
eff_quant_dim=len(nonzero_eigvals)

n_zero_eigval=n_parameters-eff_quant_dim

print("Hilbert space", 2**n_qubits)
print("Number parameters of circuit",n_parameters)
print("Effective quantum dimension G_C",eff_quant_dim)


Hilbert space 16
Number parameters of circuit 12
Effective quantum dimension G_C 10


Number of parameters of circuit that are redundant

In [9]:

#fraction of zero eigenvalues
redundancy=n_zero_eigval/n_parameters
print("redundancy",redundancy)

redundancy 0.16666666666666666


Logarithm of eigenvalues has peak when the parameter dimension become maximal. Increase depth of circuit to observe this effect.

In [10]:
qfi_var_log_eigval=np.var(np.log10(nonzero_eigvals))

print("Logarithm of non-zero qfi eigenvalues",qfi_var_log_eigval)

Logarithm of non-zero qfi eigenvalues 0.14795982881844225


Gradient and natural gradient. Calculates variance, which decreases when increasing depth of circuit and number of qubits, which is hallmark of barren plateau (or vanishing gradient) problem. Note that variance of quantum natural gradient is larger than regular gradient, but both suffer from barren plateaus. 
When observing peak in logarithm of eigenvalues, the variance of the quantum natural gradient will also decrease also

In [11]:
#inverse for quantum natural gradient
qfi_inv_matrix=scipy.linalg.pinv(qfi_matrix)
#quantum natural gradient, is gradient with quantum geometric information applied, moves efficient in parameter space
quantum_natural_gradient=np.dot(qfi_inv_matrix,gradient_list)


mean_gradient=np.mean(gradient_list)
mean_qng=np.mean(quantum_natural_gradient)
variance_gradient=np.var(gradient_list)
variance_qng=np.var(quantum_natural_gradient)

#mean of gradients and qng
print("mean gradient",mean_gradient,"mean qng",mean_qng)
#variance of gradients and qng
print("variance gradient",variance_gradient,"variance qng",variance_qng)

mean gradient 0.01929684128584129 mean qng 0.03916178538364512
variance gradient 0.0040960489197204245 variance qng 0.11806248285049863
